In [2]:
import pandas as pd
from sqlalchemy import create_engine, text, inspect,Column, Integer, String, MetaData, Table, Float,ForeignKey
from sqlalchemy.orm import sessionmaker,relationship
from sqlalchemy.exc import IntegrityError
from sqlalchemy.ext.declarative import declarative_base

import ast
import pandas as pd
import json

In [3]:
with open('secret/database_credentials.json', 'r') as f:
    db = json.load(f)
## Display the keys of the loaded dict
db.keys()

dict_keys(['mysql_username', 'mysql_password', 'mysql_host'])

In [4]:
#Database connection
mysql_username = db['mysql_username']
mysql_password = db['mysql_password']
mysql_host = db['mysql_host']
database_name = 'movies'

# Creating a connection string
connection_string = f"mysql+mysqlconnector://{mysql_username}:{mysql_password}@{mysql_host}/{database_name}"

# Creating a SQLAlchemy engine
engine = create_engine(connection_string)

# Creating a connection
connection = engine.connect()

In [84]:
# Create table title_basics
create_title_basics_table_query = '''
CREATE TABLE title_basics (
    tconst VARCHAR(255) PRIMARY KEY,
    primary_title VARCHAR(255),
    start_year INT,
    runtime_minutes INT
);
'''
connection.execute(text(create_title_basics_table_query))


In [85]:
# Create table title_ratings
create_title_ratings_table_query = '''
CREATE TABLE title_ratings (
    rating_id INT AUTO_INCREMENT PRIMARY KEY,
    tconst VARCHAR(255),
    FOREIGN KEY (tconst) REFERENCES title_basics(tconst),
    average_rating FLOAT,
    num_votes INT
);
'''
connection.execute(text(create_title_ratings_table_query))

In [86]:
# Create table genres
create_genres_table_query = '''
CREATE TABLE genres (
    genre_id INT AUTO_INCREMENT PRIMARY KEY,
    genre_name VARCHAR(255) NOT NULL
);
'''
connection.execute(text(create_genres_table_query))

In [87]:
# Create table title_genres
#The combination of title_id and genre_id is set as the primary key for the linking table.
create_title_genres_table_query = '''
CREATE TABLE title_genres (
    title_id VARCHAR(255) REFERENCES title_basics(tconst),
    genre_id INT REFERENCES genres(genre_id),
    PRIMARY KEY (title_id, genre_id)
);
'''
connection.execute(text(create_title_genres_table_query))

In [88]:
# Create table title_ratings
create_title_ratings_table_query = '''
ALTER TABLE title_genres DROP PRIMARY KEY;

ALTER TABLE title_genres
  ADD FOREIGN KEY (title_id) REFERENCES titles(tconst),
  ADD FOREIGN KEY (genre_id) REFERENCES genres(genre_id);
'''
connection.execute(text(create_title_ratings_table_query))

In [91]:
# Create table tmdb_data
create_tmdb_data_table_query = '''
CREATE TABLE tmdb_data (
    tmdb_id INT AUTO_INCREMENT PRIMARY KEY,
    tconst VARCHAR(255),
    revenue DECIMAL(15, 2),
    budget DECIMAL(15, 2),
    mpaa_rating VARCHAR(10),
    FOREIGN KEY (tconst) REFERENCES title_basics(tconst)
);
'''
connection.execute(text(create_tmdb_data_table_query))

In [92]:
# Create an Inspector instance
inspector = inspect(engine)

# Get a list of table names
table_names = inspector.get_table_names()

# Iterate through the tables and print details
for table_name in table_names:
    print(f"Table: {table_name}")
    columns = inspector.get_columns(table_name)
    for column in columns:
        print(f"  {column['name']} - {column['type']}")
    print("\n")

Table: genres
  genre_id - INTEGER
  genre_name - VARCHAR(255)


Table: title_basics
  tconst - VARCHAR(255)
  primary_title - VARCHAR(255)
  start_year - INTEGER
  runtime_minutes - INTEGER


Table: title_genres
  title_id - VARCHAR(255)
  genre_id - INTEGER


Table: title_ratings
  rating_id - INTEGER
  tconst - VARCHAR(255)
  average_rating - FLOAT
  num_votes - INTEGER


Table: tmdb_data
  tmdb_id - INTEGER
  tconst - VARCHAR(255)
  revenue - DECIMAL(15, 2)
  budget - DECIMAL(15, 2)
  mpaa_rating - VARCHAR(10)




# Now we populate our tables

In [5]:
# Load the CSV into a DataFrame
csv_path = 'tmdb_results_combined.csv'
df = pd.read_csv(csv_path, sep=';')
# Remove duplicates based on the 'tconst' column
df = df.drop_duplicates(subset=['tconst'])

In [6]:
# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

In [95]:


# Clean up the start_year column
df['start_year'] = pd.to_datetime(df['release_date'], errors='coerce').dt.year

# Convert start_year to just the year format
df['start_year'] = df['start_year'].astype('Int64')


insert_data_query = '''
    INSERT INTO title_basics (tconst, primary_title, start_year, runtime_minutes)
    VALUES (:tconst, :primary_title, :start_year, :runtime_minutes)
'''


for index, row in df.iterrows():
    # Skip rows where 'tconst' is null or NaN
    if pd.notna(row['tconst']):
        data = {
            'tconst': row['tconst'],
            'primary_title': row['title'],
            'start_year': row['start_year'] if pd.notna(row['start_year']) else None,
            'runtime_minutes': int(row['runtime']) if pd.notna(row['runtime']) else None,
        }
        try:
            connection.execute(text(insert_data_query).bindparams(**data))
        except IntegrityError as e:
            # Handle duplicate entry error by skipping the record
            continue

# Commit the changes to the database
session.commit()

# Define the SQL query to select the first 5 rows
select_query = 'SELECT * FROM title_basics LIMIT 5'

# Execute the query
result = connection.execute(text(select_query))

# Fetch and print the result
for row in result:
    print(row)

('tt0035423', 'Kate & Leopold', 2001, 118)
('tt0113026', 'The Fantasticks', 2000, 86)
('tt0113092', 'For the Cause', 2000, 100)
('tt0118589', 'Glitter', 2001, 104)
('tt0118652', 'The Attic Expeditions', 2001, 100)


In [96]:
insert_data_query = '''
    INSERT INTO title_ratings (tconst, average_rating, num_votes)
    VALUES (:tconst, :average_rating, :num_votes)
'''


for index, row in df.iterrows():
    # Skip rows where 'tconst' is null or NaN
    if pd.notna(row['tconst']):
        data = {
            'tconst': row['tconst'],
            'average_rating': row['vote_average'],
            'num_votes': row['vote_count'],
        }
        try:
            connection.execute(text(insert_data_query).bindparams(**data))
        except IntegrityError as e:
            # Handle duplicate entry error by skipping the record
            continue

# Commit the changes to the database
session.commit()

# Define the SQL query to select the first 5 rows
select_query = 'SELECT * FROM title_ratings LIMIT 5'

# Execute the query
result = connection.execute(text(select_query))

# Fetch and print the result
for row in result:
    print(row)

(1, 'tt0175142', 6.3, 6446)
(2, 'tt0120917', 7.6, 6163)
(3, 'tt0195714', 6.6, 5158)
(4, 'tt0130623', 6.5, 2312)
(5, 'tt0120755', 6.1, 6205)


In [97]:
# Function to safely convert string representation of lists to actual lists
def safe_literal_eval(value):
    try:
        return ast.literal_eval(value)
    except (SyntaxError, ValueError):
        return value

# Apply the safe literal eval function to the 'genres' column
df['genres'] = df['genres'].apply(safe_literal_eval)

# Extracting genres and their ids from the 'genres' column
all_genres_with_id = [(genre['id'], genre['name']) for genres_list in df['genres'] for genre in genres_list if isinstance(genre, dict)]

# Remove duplicates by converting the list to a set and then back to a list
unique_genres_with_id = list(set(all_genres_with_id))

# Convert the list to the desired format
unique_genres_dict = {genre[0]: genre[1] for genre in unique_genres_with_id}

# Define the Genre table
metadata = MetaData()
genre_table = Table('genres', metadata,
                   Column('genre_id', Integer, primary_key=True),
                   Column('genre_name', String)
                   )

# Create the table (if it doesn't exist)
metadata.create_all(engine)

# Use a session to insert data into the table
Session = sessionmaker(bind=engine)
session = Session()

# Insert unique genres into the 'genre' table
for genre_id, genre_name in unique_genres_dict.items():
    session.execute(genre_table.insert().values(genre_id=genre_id, genre_name=genre_name))

# Commit the changes
session.commit()

In [40]:
# Define the SQL query to select the first 5 rows
select_query = 'SELECT * FROM genres LIMIT 5'


try:
    result = connection.execute(text(select_query))

    # Fetch and print the result
    for row in result:
        print(row)

except Exception as e:
    print(f"Error: {e}")

(12, 'Adventure')
(14, 'Fantasy')
(16, 'Animation')
(18, 'Drama')
(27, 'Horror')


In [98]:

# Function to extract genre ids
def extract_genre_ids(genre_list):
    return [genre['id'] for genre in genre_list]

# Apply the function to create a new 'genre_ids' column
df['genre_ids'] = df['genres'].apply(extract_genre_ids)

# Display the DataFrame
print(df[['tconst', 'genre_ids']])

# Explode the 'genre_ids' column to create individual rows for each genre ID
df_exploded = df.explode('genre_ids')

df_exploded.rename(columns={'tconst': 'title_id', 'genre_ids': 'genre_id'}, inplace=True)

# Drop rows where 'title_id' is NULL
df_exploded_cleaned = df_exploded.dropna(subset=['title_id'])

# Insert data into the title_genres table
df_exploded_cleaned[['title_id', 'genre_id']].to_sql('title_genres', con=engine, if_exists='append', index=False)





           tconst                genre_ids
0       tt0175142                     [35]
1       tt0120917  [12, 16, 35, 10751, 14]
2       tt0195714                     [27]
3       tt0130623              [10751, 16]
4       tt0120755             [12, 28, 53]
...           ...                      ...
173757  tt0277713                     [35]
174899  tt0304628                     [18]
176042  tt0261236                 [18, 35]
177186  tt0278443                     [35]
178331  tt0284470                     [18]

[1119 rows x 2 columns]


In [7]:
# Define the SQL query to select the first 5 rows
select_query = 'SELECT * FROM title_genres LIMIT 10'


try:
    result = connection.execute(text(select_query))

    # Fetch and print the result
    for row in result:
        print(row)

except Exception as e:
    print(f"Error: {e}")

('tt0175142', 35)
('tt0120917', 12)
('tt0120917', 16)
('tt0120917', 35)
('tt0120917', 10751)
('tt0120917', 14)
('tt0195714', 27)
('tt0130623', 10751)
('tt0130623', 16)
('tt0120755', 12)


In [99]:
# Select only the required columns
selected_columns = ['tconst', 'revenue', 'budget', 'mpaa_rating']
df_selected = df[selected_columns]

# Specify the table name (tmdb_data) and the engine
table_name = 'tmdb_data'

# Use the to_sql method to insert data into the table
df_selected.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

In [69]:
# Define the SQL query to select the first 5 rows
select_query = 'SELECT * FROM tmdb_data LIMIT 10'


try:
    result = connection.execute(text(select_query))

    # Fetch and print the result
    for row in result:
        print(row)

except Exception as e:
    print(f"Error: {e}")

('tt0175142', 278019771, 19000000, 'R')
('tt0120917', 169327687, 100000000, 'G')
('tt0195714', 112880294, 23000000, None)
('tt0130623', 354248063, 127500000, 'PG')
('tt0120755', 546388105, 125000000, 'PG-13')
('tt0134084', 161834276, 40000000, 'R')
('tt0138749', 76432727, 95000000, 'PG')
('tt0209144', 39723096, 9000000, 'R')
('tt0187078', 237202299, 90000000, 'PG-13')
('tt0162222', 429632142, 90000000, 'PG-13')
